In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, utils
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import MobileNetV2
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels_cat = utils.to_categorical(train_labels, 10)
test_labels_cat = utils.to_categorical(test_labels, 10)

# Model 1: Basic CNN
def define_model_1():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 2: CNN with Transfer Learning
def define_model_2():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=layers.Input(shape=(96, 96, 3)))
    base_model.trainable = False
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Lambda(lambda image: tf.image.grayscale_to_rgb(image))(inputs)
    x = layers.Resizing(96, 96)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 3: Simple RNN
def define_model_3():
    model = models.Sequential([
        layers.SimpleRNN(128, input_shape=(28, 28), return_sequences=True),
        layers.SimpleRNN(64),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 4: RNN with Transfer Learning
def define_model_4():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3), pooling='avg')
    base_model.trainable = False
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Lambda(lambda image: tf.image.grayscale_to_rgb(image))(inputs)
    x = layers.Resizing(96, 96)(x)
    x = base_model(x, training=False)
    x = layers.RepeatVector(3)(x)
    x = layers.SimpleRNN(128, return_sequences=False)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Dictionary of models
models_dict = {
    'model_1': define_model_1(),
    'model_2': define_model_2(),
    'model_3': define_model_3(),
    'model_4': define_model_4(),

}

# Train and evaluate models
conf_matrices = {}
for name, model in models_dict.items():
    print(f"Training {name}...")
    if '3' in name or '4' in name:
        model.fit(train_images, train_labels_cat, epochs=10, validation_split=0.1)
    else:
        model.fit(train_images, train_labels, epochs=10, validation_split=0.1)
    test_preds = np.argmax(model.predict(test_images), axis=1)
    cm = confusion_matrix(test_labels, test_preds)
    conf_matrices[name] = cm
    print(f"{name} Confusion Matrix:\n", cm)

# Dynamic ensemble voting based on class-specific accuracies
threshold = 0.85
votes = np.zeros((len(test_images), 10), dtype=int)

for i in range(len(test_images)):
    model_votes = {name: np.argmax(model.predict(test_images[i:i+1]), axis=1)[0]
                   for name, model in models_dict.items()}
    for label in range(10):
        for name, pred in model_votes.items():
            if conf_matrices[name][label][label] / np.sum(conf_matrices[name][label]) > threshold:
                votes[i][pred] += 1

final_predictions = np.argmax(votes, axis=1)
final_accuracy = np.mean(final_predictions == test_labels)
print(f"Final Ensemble Accuracy: {final_accuracy}")

# Plot final confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(test_labels, final_predictions), annot=True, fmt="d", cmap='Blues')
plt.title('Final Ensemble Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()
